In [1]:
import sqlite3

User             DBMS               database
python          sqlite3  <--SQL-->    FILE
     connection
       cursor

In [5]:
con = sqlite3.connect(':memory:')

In [6]:
cur = con.cursor()

In [7]:
cur.excute #sql 1개 실행
cur.executemany #sql 1개를 n번 반복 실행
cur.excutescript #(비표준) n개의 sql문 실행

AttributeError: 'sqlite3.Cursor' object has no attribute 'excute'

In [8]:
cur.execute('''
    CREATE TABLE TEST(
        ID INTEGER,
        NAME CHAR(1)
        );
''')

In [9]:
cur.execute('''
    INSERT INTO TEST (ID, NAME) VALUES (1,'A');    
''')

In [10]:
cur.execute('select * from test;')

In [11]:
cur.execute('''
    INSERT INTO TEST (ID, NAME) VALUES (1,'ABC');
''')

In [12]:
cur.execute('select * from test;')

In [13]:
cur.fetchone() #1개
cur.fetchmany(2) #2개
cur.fetchall() # 전체

#fetch는 내 컴퓨터에서 실행시키는 것이다

<function Cursor.fetchall()>

In [14]:
cur.fetchall()

[(1, 'A'), (1, 'ABC')]

In [18]:
cur.execute('INSERT INTO TEST VALUES (3,NULL);')

In [20]:
cur.execute('select * from test')
cur.fetchall()

[(1, 'A'), (1, 'ABC'), (3, None)]

In [21]:
con.close() # 작업 끝나면 닫아줘야한다

In [2]:
con = sqlite3.connect('coffee.db')
cur = con.cursor()

In [3]:
cur.execute('drop table city;')

In [4]:
con.execute('''
    create table city(
        cno integer primary key,
        name text not null
    );
''')

In [5]:
con.execute('''
    insert into city (cno, name) values (1, '성북구');
''')

In [6]:
con.execute('''
    insert into city (cno, name) values (2, '강북구');
''')

In [7]:
con.execute('''
    insert into city (cno, name) values (null, '동대문구');
''')

In [8]:
con.execute('''
    insert into city (cno, name) values (3, '노원구');
''')

IntegrityError: UNIQUE constraint failed: city.cno

In [9]:
cur.execute('select * from city;')
cur.fetchall()

[(1, '성북구'), (2, '강북구'), (3, '동대문구')]

In [10]:
cur.lastrowid

3

db browser for sqlite # 숙제
=> table 구조 뜨는거 확인하고, 이상한점 찾기 (데이터가 안들어가있음)

      sql
DBMS   =>  DB
--------------
TRANSACTION
데이터 추가/수정(갱신) => 변화가 있을 때
commit

In [11]:
con.commit()

In [12]:
con.execute('insert into city values (null, ?);', ('중구',))

In [13]:
type('중구'), type(('중구')), type(('중구',))

(str, str, tuple)

In [14]:
cur.execute('select * from city;')
cur.fetchall()

[(1, '성북구'), (2, '강북구'), (3, '동대문구'), (4, '중구')]

In [15]:
con.execute('insert into city values (null, :name);', {'name' : '중량구'})

In [16]:
cur.executemany('''
    insert into city (name) values (?)
''', [('노원구',), ('강남구',), ('서초구',)])

In [17]:
cur.execute('select * from city;')
cur.fetchall()

[(1, '성북구'),
 (2, '강북구'),
 (3, '동대문구'),
 (4, '중구'),
 (5, '중량구'),
 (6, '노원구'),
 (7, '강남구'),
 (8, '서초구')]

In [18]:
cur.executescript('''
    drop table if exists supplier;
    create table supplier(
        sno integer primary key,
        name varchar(2),
        cno integer integer not null
        );
    drop table if exists part;
    create table part(
        pno integer primary key,
        name char(1)
        );
''')

serverless (direct I/O) => FILE 1개만 open


connect 여러번 x => lock

In [19]:
p = [{'n':'아메리카노'}, {'n':'카페라테'}, {'n':'에스프레소'}, {'n':'아이스티'}]
# len(p) == 4, p[0] == {'n':'아메리카노'}, p[0]['n'] == 아메리카노
cur.executemany('insert into part (name) values (:n)', p) # 4번 반복, sql문 1개, 각 반복마다 n키값에 대응되는 vlaue

In [20]:
cur.execute('select * from part;')
cur.fetchall()

[(1, '아메리카노'), (2, '카페라테'), (3, '에스프레소'), (4, '아이스티')]

cursor.execute 함수들




1. python 파라미터 2개 넘김(sql, 변수) # <= 여기


2. dbms SQL 해석


3. db SQL 적용

In [21]:
cur.execute('select cno from city where name=?', ['성북구'])
cur.fetchall()

[(1,)]

In [22]:
cur.execute('select cno from city where name like ?', ['%구'])
cur.fetchall()

[(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)]

In [23]:
cur.execute('''
    select * from city where name like ?
    order by name, cno
''', ['%구'])
cur.fetchall()

[(7, '강남구'),
 (2, '강북구'),
 (6, '노원구'),
 (3, '동대문구'),
 (8, '서초구'),
 (1, '성북구'),
 (4, '중구'),
 (5, '중량구')]

In [24]:
# DBMS
# limit 시작위치, 몇개

cur.execute('''
    select * from city where name like ?
    order by name, cno asc
    limit 0, 2
''', ['%구'])
cur.fetchall()

[(7, '강남구'), (2, '강북구')]

In [28]:
cur.execute('''
    select * from city where name like ?
    order by name, cno asc
    limit 0, 1
''', ['%성북구%'])
# cursor => dbms <- db
cur.fetchone() # record/tuple/row 단위로 가져옴 => python iterable 객체로 변환
# cursor 결과 => 읽어들임
cur.fetchone()[0]
# cursor의 결과의 결과를 읽으니 에러

TypeError: 'NoneType' object is not subscriptable

In [27]:
cur.execute('''
    select * from city where name like ?
    order by name, cno asc
    limit 0, 1
''', ['%성북구%'])
cno = cur.fetchone()[0]
print(cno)

1


In [29]:
# 보통은 이렇게 중첩시키고 함수(프로시저)를 만든다
# 프로시저를 쓰면 보안에 좋고 여러가지 작업을 한번에 할 수 있다
cur.execute('insert into supplier (name, cno) values(?,?)', ('1호점', cno))
cur.execute('insert into supplier (name, cno) values(?,?)', ('2호점', cno))
cur.execute('insert into supplier (name, cno) values(?,?)', ('3호점', cno))

In [30]:
cur.execute('select * from supplier')
cur.fetchall()

[(1, '1호점', 1), (2, '2호점', 1), (3, '3호점', 1)]

cur.execute('''


    insert into supplier (name, cno) values (?, ? <= 요기만)

    
''')

In [31]:
cur.execute('''
    insert into supplier (name, cno) values (?, (
        select cno from city where name like ?
        order by name, cno asc
        limit 0, 1
    ))
''', ('1호점', '%강북구%'))

In [32]:
cur.execute('select name from city where cno > 2')
c = cur.fetchall()

for name in c:
    name = '%' + name[0] + '%'

    # for no in ['1호점', '2호점', '3호점']:
    #     cur.execute('''
    #         insert into supplier (name, cno) values (?, (
    #             select cno from city where name like ?
    #             order by name, cno asc
    #             limit 0, 1
    #         ))
    #     ''', (no, name))
    params = [{'no':'1호점', 'name':name}, {'no':'2호점', 'name':name}, {'no':'3호점', 'name':name}]
    cur.executemany('''
        INSERT INTO SUPPLIER (NAME, CNO) VALUES (:no, (
            SELECT CNO FROM CITY WHERE NAME LIKE :name
            ORDER BY NAME, CNO ASC
            LIMIT 0, 1
        ))
    ''', params)

In [33]:
cur.execute('select * from supplier')
len(cur.fetchall())

22

In [34]:
cur.execute('select count(pno) from part')
cur.fetchone()[0]

4

In [35]:
cur.executescript('''
    drop table if exists sells;
    create table sells(
        sno ineger not null,
        pno integer not null,
        price ineger not null default 0
    )
''')

In [36]:
cur.executemany('insert into sells values (?, ?, ?)', [(1,1,10),
(2,1,10),
(3,1,10),
(5,1,10),
(7,1,10),
(9,1,10),
(11,1,10),
(1,2,15),
(4,2,15),
(6,2,15),
(8,2,15),
(10,2,15),
(12,2,15),
(14,2,15),
(1,3,8),
(18,3,8),
(19,3,8),
(22,4,8),
(1,4,20),
(2,4,20),
(3,4,20)])

In [48]:
cur.execute('select * from sells')
cur.fetchall()

[(1, 1, 10),
 (2, 1, 10),
 (3, 1, 10),
 (5, 1, 10),
 (7, 1, 10),
 (9, 1, 10),
 (11, 1, 10),
 (1, 2, 15),
 (4, 2, 15),
 (6, 2, 15),
 (8, 2, 15),
 (10, 2, 15),
 (12, 2, 15),
 (14, 2, 15),
 (1, 3, 8),
 (18, 3, 8),
 (19, 3, 8),
 (22, 4, 8),
 (1, 4, 20),
 (2, 4, 20),
 (3, 4, 20)]

In [37]:
con.commit()

In [38]:
cur.execute('select name from city')
cur.fetchall()

[('성북구',),
 ('강북구',),
 ('동대문구',),
 ('중구',),
 ('중량구',),
 ('노원구',),
 ('강남구',),
 ('서초구',)]

In [39]:
cur.execute('insert into city (name) values("광진구");')

In [40]:
cur.execute('''
    select city.name, supplier.name from city
    inner join supplier
    on city.cno = supplier.cno
''')
len(cur.fetchall())

22

In [41]:
cur.execute('''
    select city.name, supplier.name from city
    left join supplier
    on city.cno = supplier.cno
''')
len(cur.fetchall())

23

In [42]:
cur.execute('''
    select city.name, count(supplier.name) from city
    left join supplier
    on city.cno = supplier.cno
    group by city.cno
    order by city.name asc
''')

cur.fetchall()

[('강남구', 3),
 ('강북구', 1),
 ('광진구', 0),
 ('노원구', 3),
 ('동대문구', 3),
 ('서초구', 3),
 ('성북구', 3),
 ('중구', 3),
 ('중량구', 3)]

In [43]:
cur.execute('''
    select city.name, supplier.name, count(supplier.name), sum(sells.price) from city
    left join supplier
    on supplier.cno = city.cno
    inner join sells
    on supplier.sno = sells.sno
    group by city.name, supplier.name
    order by city.cno asc
''')

cur.fetchall()

[('성북구', '1호점', 4, 53),
 ('성북구', '2호점', 2, 30),
 ('성북구', '3호점', 2, 30),
 ('강북구', '1호점', 1, 15),
 ('동대문구', '1호점', 1, 10),
 ('동대문구', '2호점', 1, 15),
 ('동대문구', '3호점', 1, 10),
 ('중구', '1호점', 1, 15),
 ('중구', '2호점', 1, 10),
 ('중구', '3호점', 1, 15),
 ('중량구', '1호점', 1, 10),
 ('중량구', '2호점', 1, 15),
 ('노원구', '1호점', 1, 15),
 ('강남구', '2호점', 1, 8),
 ('강남구', '3호점', 1, 8),
 ('서초구', '3호점', 1, 8)]

In [44]:
cur.execute('''
    select city.name, part.name, count(part.name), sum(sells.price) from city
    left join supplier
    on supplier.cno = city.cno
    inner join sells
    on supplier.sno = sells.sno
    inner join part
    on part.pno = sells.pno
    group by city.name, part.name
    order by city.cno asc
''')

cur.fetchall()

[('성북구', '아메리카노', 3, 30),
 ('성북구', '아이스티', 3, 60),
 ('성북구', '에스프레소', 1, 8),
 ('성북구', '카페라테', 1, 15),
 ('강북구', '카페라테', 1, 15),
 ('동대문구', '아메리카노', 2, 20),
 ('동대문구', '카페라테', 1, 15),
 ('중구', '아메리카노', 1, 10),
 ('중구', '카페라테', 2, 30),
 ('중량구', '아메리카노', 1, 10),
 ('중량구', '카페라테', 1, 15),
 ('노원구', '카페라테', 1, 15),
 ('강남구', '에스프레소', 2, 16),
 ('서초구', '아이스티', 1, 8)]

In [45]:
# sql 중첩 => procedure
# join(inner = 교집합, left/right = 기준테이블, outer)
# group by

In [ ]:
#dump는 db 백업할때 쓴다

In [46]:
for d in con.iterdump():
    print(d)

BEGIN TRANSACTION;
CREATE TABLE city(
        cno integer primary key,
        name text not null
    );
INSERT INTO "city" VALUES(1,'성북구');
INSERT INTO "city" VALUES(2,'강북구');
INSERT INTO "city" VALUES(3,'동대문구');
INSERT INTO "city" VALUES(4,'중구');
INSERT INTO "city" VALUES(5,'중량구');
INSERT INTO "city" VALUES(6,'노원구');
INSERT INTO "city" VALUES(7,'강남구');
INSERT INTO "city" VALUES(8,'서초구');
INSERT INTO "city" VALUES(9,'광진구');
CREATE TABLE part(
        pno integer primary key,
        name char(1)
        );
INSERT INTO "part" VALUES(1,'아메리카노');
INSERT INTO "part" VALUES(2,'카페라테');
INSERT INTO "part" VALUES(3,'에스프레소');
INSERT INTO "part" VALUES(4,'아이스티');
CREATE TABLE sells(
    sno ineger not null,
    pno integer not null,
    price ineger not null default 0
    );
INSERT INTO "sells" VALUES(1,1,10);
INSERT INTO "sells" VALUES(2,1,10);
INSERT INTO "sells" VALUES(3,1,10);
INSERT INTO "sells" VALUES(5,1,10);
INSERT INTO "sells" VALUES(7,1,10);
INSERT INTO "sells" VALUES(9,1,10);
INSERT 